In [1]:
import sys
import itertools

import gurobipy as gb

In [2]:
# define target cells and allowed values
cells = [
           (2,3), (2,4),
    (3,2), (3,3), (3,4), (3,5), (3,6),
    (4,2), (4,3),        (4,5), (4,6),
    (5,2), (5,3), (5,4), (5,5), (5,6),
                  (6,4), (6,5)
]
values = list(range(1,10))

In [3]:
# the given 
data_horizontal = {
    3: [[(2,3), (2,4)], 
        [(4,2), (4,3)]],
    16: [[(3,2), (3,3), (3,4), (3,5), (3,6)],
         [(6,4), (6,5)]],
    13: [[(4,5), (4,6)]],
    17: [[(5,2), (5,3), (5,4), (5,5), (5,6)]]
}

data_vertical = {
    7: [[(3,2), (4,2), (5,2)]],
    10: [[(2,3), (3,3), (4,3), (5,3)]],
    4: [[(2,4), (3,4)]],
    9: [[(5,4), (6,4)]],
    30: [[(3,5), (4,5), (5,5), (6,5)]],
    8: [[(3,6), (4,6), (5,6)]]
}

In [4]:
# declare and initialize model
m = gb.Model('kakuro')

Using license file /home/faustind/gurobi.lic
Academic license - for non-commercial use only


In [5]:
# add decision variables for cell values
assign = m.addVars(cells, values, name="assign", vtype=gb.GRB.BINARY)

In [6]:
# a single cell cannot be assigned more than one value
unique_assignement = m.addConstrs((assign.sum(x, y, '*') == 1
                                  for x, y in cells), 'unique_assignment')

In [7]:
# horizontal clues constraints
for total, lists in data_horizontal.items():
    for i, l in enumerate(lists):
        expr = gb.LinExpr()
        for row, col in l:
            for v in values:
                expr.add(assign[row, col, v]*v)
        m.addConstr(expr == total, "hor_{}_{}".format(total, i))

In [8]:
# vertical clues constraints
for total, lists in data_vertical.items():
    for i, l in enumerate(lists):
        expr = gb.LinExpr()
        for row, col in l:
            for v in values:
                expr.add(assign[row, col, v]*v)
        m.addConstr(expr == total, "vert_{}_{}".format(total, i))

In [9]:
# constraint for unicity of values in the same group

for _, lists in data_vertical.items():
    for l in lists:
        for (x1, y1), (x2, y2) in itertools.combinations(l, 2):
            m.addConstrs(assign[x1, y1, v] + assign[x2, y2, v] <= 1 
                        for v in values)
            
for _, lists in data_horizontal.items():
    for l in lists:
        for (x1, y1), (x2, y2) in itertools.combinations(l, 2):
            m.addConstrs(assign[x1, y1, v] + assign[x2, y2, v] <= 1 
                        for v in values)

In [10]:
m.setObjective(1)

In [11]:
m.write('files/kakuro.lp')

In [12]:
m.optimize()

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (linux64)
Optimize a model with 426 rows, 162 columns and 1278 nonzeros
Model fingerprint: 0x5640372b
Variable types: 0 continuous, 162 integer (162 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Presolve removed 426 rows and 162 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds
Thread count was 1 (of 4 available processors)

Solution count 1: 1 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.000000000000e+00, best bound 1.000000000000e+00, gap 0.0000%


In [13]:
# display optimal values of decision variables
for v in m.getVars():
    if (abs(v.x) > sys.float_info.epsilon):
        print(v.varName, v.x)

assign[2,3,2] 1.0
assign[2,4,1] 1.0
assign[3,2,1] 1.0
assign[3,3,4] 1.0
assign[3,4,3] 1.0
assign[3,5,6] 1.0
assign[3,6,2] 1.0
assign[4,2,2] 1.0
assign[4,3,1] 1.0
assign[4,5,8] 1.0
assign[4,6,5] 1.0
assign[5,2,4] 1.0
assign[5,3,3] 1.0
assign[5,4,2] 1.0
assign[5,5,7] 1.0
assign[5,6,1] 1.0
assign[6,4,7] 1.0
assign[6,5,9] 1.0
total matching score 1.0
